Домашнее задание
Продвинутый Pandas 2

**Задание 1**

1. Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.


In [2]:
import pandas as pd

In [3]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')

In [7]:
# оставим колонки userId и rating, сгруппируем таблицу про userId и посчитаем количество выставленных оценок полльзователем
grouped = ratings[['userId','rating']].groupby('userId').count().reset_index()

# отфильтруем таблицу выше по пользователям поставившим более 100 оценок
filter_users = grouped[['userId']].loc[grouped.rating > 100]

# создадим новую таблицу, сгруппируем по userId и посчитаес макс и мин значение по столбцам
users_grouped = ratings.groupby('userId').agg([min, max])

# создадим новый столбец который будет содержать разницу между макс и мин значением столбца timestamp
users_grouped['difference'] = users_grouped['timestamp']['max'] - users_grouped['timestamp']['min']

# переведем значение столбца difference из секунд в дни
users_grouped['difference'] = users_grouped['difference'].apply(lambda x: x/24/3600)

# объединим таблицы выше по userId и оставим один столбец difference для наглядности 
filter_users.join(users_grouped['difference'], how='left')

,userId,difference
3,4,0.824051
7,8,0.029954
14,15,0.007859
16,17,472.075752
18,19,0.986690
...,...,...
655,656,0.065069
658,659,0.017454
663,664,0.174606
664,665,1136.347674


**Задание  2**

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.


Дана статистика услуг перевозок клиентов компании по типам:

rzd - железнодорожные перевозки
auto - автомобильные перевозки
air - воздушные перевозки
client_base - адреса клиентов

In [8]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [9]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [10]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [11]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [49]:
table = (
    client_base
    .merge(rzd, how='left')
    .merge(auto, how='left')
    .merge(air, how='left'))


(
table
.pivot_table(index=['client_id'], values=['rzd_revenue','auto_revenue','air_revenue'], fill_value=0)
.reset_index()
)


,client_id,air_revenue,auto_revenue,rzd_revenue
0,111,0,0,1093
1,112,0,0,2810
2,113,0,57483,10283
3,114,0,83,5774
4,115,81,912,981
5,116,4,4834,0
6,117,13,98,0
7,118,173,0,0


In [50]:
table = (
    client_base
    .merge(rzd, how='left')
    .merge(auto, how='left')
    .merge(air, how='left'))

(
table
.pivot_table(index=['client_id','address'], values=['rzd_revenue','auto_revenue','air_revenue'], fill_value=0)
.reset_index()
)

,client_id,address,air_revenue,auto_revenue,rzd_revenue
0,111,Комсомольская 4,0,0,1093
1,112,Энтузиастов 8а,0,0,2810
2,113,Левобережная 1а,0,57483,10283
3,114,Мира 14,0,83,5774
4,115,ЗЖБИиДК 1,81,912,981
5,116,Строителей 18,4,4834,0
6,117,Панфиловская 33,13,98,0
7,118,Мастеркова 4,173,0,0


**Задание 3**

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
* У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
* Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
* Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.


нет опыта работы в аналитике, возможно буду писать чушь.

По поводу добавления сведений в таблицу:
для связи визита и покупки дописать в конце данных координат число, условно 1 - купил, 0 - зашел посмотреть. 
Возможно потребуется перед координатами дописать дату. 
Либо создать 3 столбца, дата, статус(купил/посмотрел), координаты.

Что можно получить из координат? наиболее часто встречающиеся места будут скорее всего дом и работа, проверив на картах, что там находится, можно с высокой долей вероятности это определить. Если посмотреть в каких местах пользователь только заходит, а в каких покупает, сколько времени между этими промежутками, возможно получится сократить этот промежуток, прислав уведомление или письмо ускорив принятие им решения.
